## Initial Set-Up

We'll be using the [resize (cv2) function](https://www.opencv.org.cn/opencvdoc/2.3.2/html/modules/imgproc/doc/geometric_transformations.html#cv2.resize)

In [2]:
!pip3.10 install opencv-python
!pip3.10 install tqdm

You should consider upgrading via the '/Users/saifalesawy/.pyenv/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saifalesawy/.pyenv/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.


In [3]:
import cv2
import os
import numpy as np
from tqdm import tqdm # progress bar

In [5]:
directory = "../mirflickr"
count = 0
# iterate through each file in the dir and push file data into imgs list
for filename in tqdm(os.listdir(directory)):
    filepath = os.path.join(directory, filename)
    img = cv2.imread(filepath)

    if '.jpg' not in filename:
        continue
    
    try:
        hr_output = './imgs-hr/' + filename
        lr_output = './imgs-lr/' + filename
        hr_img = cv2.resize(img, np.array([128, 128]))
        lr_img = cv2.resize(img, np.array([32, 32]))

        cv2.imwrite(hr_output, hr_img)
        cv2.imwrite(lr_output, lr_img)
    except Exception as e:
        print('failed to resize ', hr_output, e)
        print(filename)
        print(img)

100%|██████████| 25003/25003 [01:48<00:00, 231.42it/s]
